# Tema 2.2: Métodos, Visibilidad y Propiedades

## 1. Métodos de Instancia, Clase y Estáticos

Hasta ahora hemos visto los métodos de instancia, que reciben `self` como primer parámetro y operan sobre una instancia o objeto concreto de la clase.

Sin embargo, Python ofrece otros tipos de métodos que no operan sobre instancias concretas:

### Métodos de Clase (@classmethod)

Reciben la clase como primer parámetro (habitualmente llamado `cls`) en lugar de la instancia. 

Se utilizan para operaciones que afectan a la clase en sí, o para crear constructores alternativos (factory methods).

In [ ]:
class Fecha:
    def __init__(self, dia, mes, anyo):
        self.dia = dia
        self.mes = mes
        self.anyo = anyo

    @classmethod
    def desde_cadena(cls, fecha_str):
        # Factoría: Crea una instancia a partir de "dd-mm-yyyy"
        dia, mes, anyo = map(int, fecha_str.split('-'))
        return cls(dia, mes, anyo)

f1 = Fecha(10, 12, 2023)
f2 = Fecha.desde_cadena("25-12-2023")

print(f"Fecha 1: {f1.dia}/{f1.mes}/{f1.anyo}")
print(f"Fecha 2: {f2.dia}/{f2.mes}/{f2.anyo}")

### Métodos Estáticos (@staticmethod)
No reciben ni `self` ni `cls`. Son funciones normales que pertenecen al espacio de nombres de la clase por organización lógica, pero no acceden al estado de la instancia ni de la clase. Si necesitan información para generar la salida, esta la obtienen del "exterior": de los parámetros de la función.

In [ ]:
class Calculadora:
    @staticmethod
    def sumar(a, b):
        return a + b

print(Calculadora.sumar(5, 3))

### Diferencias entre Métodos de clase y métodos estáticos

La diferencia fundamental es que el método de clase recibe automáticamente la propia clase como argumento (cls), lo que le permite acceder y modificar el estado de la clase o actuar como constructor alternativo. Por el contrario, el método estático no recibe ningún argumento implícito, comportándose como una función normal que reside dentro de la clase únicamente por organización lógica.

Ejemplo:

In [13]:
class Pizza:
    def __init__(self, ingredientes):
        self.ingredientes = ingredientes

    # Método de clase: Funciona como un constructor alternativo
    @classmethod
    def margarita(cls):
        return cls(['mozzarella', 'tomate'])

    # Método estático: Una utilidad que no necesita saber nada de la clase pizza 
    #   ni de ninguna pizza en concreto. La información que necesita la recoge del "exterior". 
    @staticmethod
    def validar_ingrediente(ingrediente):
        return ingrediente != "piña" # Debate eterno

p1 = Pizza(['nata', 'bacon', 'cebolla'])
print("Mi custom carbonara: ", p1, p1.ingredientes)

# probando método de clase
p2 = Pizza.margarita()
print("La margarita de la casa: ", p2, p2.ingredientes)

# probando método estático
brocoli_valido = Pizza.validar_ingrediente("brocoli")
print(f"Brocoli: {"sí" if brocoli_valido else "ni de broma"}")

pinya_valida = Pizza.validar_ingrediente("piña")
print(f"Piña: {"sí" if pinya_valida else "ni de broma"}")

Mi custom carbonara:  <__main__.Pizza object at 0x7a57cf0b7950> ['nata', 'bacon', 'cebolla']
La margarita de la casa:  <__main__.Pizza object at 0x7a57cf0b5370> ['mozzarella', 'tomate']
Brocoli: sí
Piña: ni de broma


## 2. Visibilidad y Encapsulamiento

En lenguajes como Java o C++, existen palabras clave como `public`, `private` o `protected` que limitan la visibilidad (acceso) a los miembros (atributos, métodos) de una clase desde fuera (p.e. desde un código principal).

En Python, **todo es público por defecto**. Esto significa que todos los miembros de una clase son accesibles desde fuera de ella y se pueden modificar libremente.

Sin embargo, existen convenciones y mecanismos que simulan la privacidad `private` y `protected`:

*   **Público**: `nombre`. 
     * Accesible desde cualquier lugar, dentro y fuera de la clase.
     * Sin restricciones para modificar el contenido (valor) de los miembros del objeto.
*   **Protegido (convención)**: `_nombre` (un guion bajo delante). 
     * Sigue siendo accesible, pero indica al programador que es de uso interno o para subclases.
*   **Privado (Name Mangling)**: `__nombre` (dos guiones bajos delante). 
     * No se puede acceder desde fuera de la clase. 
     * Motivación: limitar o controlar explícitamente la forma en la que se puede acceder o modificar el estado interno del objeto. 
     * El intérprete de Python cambia el nombre internamente a `_NombreClase__nombre` *(name mangling)* para dificultar el acceso accidental desde fuera.


De esta manera, **encapsulamos** (confinamos) la información dentro del objeto, y nos reservamos la posibilidad permitir su manipulación mediante la programación de métodos consultores y/o modificadores (ver Sección 3).

In [21]:
class CuentaBancaria:
    def __init__(self, titular, saldo):
        self.titular = titular       # Público
        self._tipo = "Ahorro"        # Protegido (convención)
        self.__saldo = saldo         # Privado. No queremos que nadie modifique libremente 
                                     #          el saldo de una cuenta bancaria

    def depositar(self, cantidad):
        if cantidad > 0:
            self.__saldo += cantidad

    def obtener_saldo(self):
        return self.__saldo

cuenta = CuentaBancaria("Júlia", 1000)

print(cuenta.titular)
print(cuenta._tipo)       # Accesible, pero no deberíamos tocarlo

# print(cuenta.__saldo)          # Error: AttributeError
print(cuenta.obtener_saldo())    # Forma correcta de acceder

# cuenta.__saldo += 1000000     # Ójala pudiéramos

print(cuenta._CuentaBancaria__saldo) # Acceso 'truculento' (Name Mangling) - NO USAR
cuenta._CuentaBancaria__saldo += 1000000  # Acabamos de hackear el banco...

print(cuenta.obtener_saldo()) 

Júlia
Ahorro
1000
1000
1001000


## 3. Métodos consultores (getters) y modificadores (setters)

Cuando definimos atributos privados, estamos restringiendo totalmente la consulta y la modificación de los valores de dichos atributos. La información queda **encapsulada** o confinada dentro del objeto.

Esto nos da la posibilidad de controlar explícitamente de qué manera se consulta o se modifica dicha información. Lo haremos mediante la definición de métodos consultores (p.e. `get_variable()`) y modificadores (p.e. `set_variable(nuevo_valor)`).

In [ ]:
class Circulo:
    def __init__(self, radio):
        self.__radio = radio # Usamos guion bajo para el atributo interno

    def get_radio(self):
        """Método consultor (Getter)"""
        print("Leyendo radio...") # Por verbosidad nada más; evitar poner prints aquí!
        return self.__radio

    def set_radio(self, valor):
        """Método modificador (Setter)"""
        if valor < 0:
            raise ValueError("El radio no puede ser negativo")
        print("Asignando radio...") # Por verbosidad nada más; evitar poner prints aquí!
        self.__radio = valor

    def area(self):
        """Método regular, calcula y devuelve el área del círculo"""
        return 3.1416 * (self.__radio ** 2)

c = Circulo(5)

#print(c.radio)      # Error!!
#print(c.__radio)    # Error!!
print(c.get_radio())

#c.radio = 10        # Error
#c.__radio = 10      # Error
c.set_radio(10)
print(c.get_radio())

print(f"Área: {c.area()}")
# c.area = 20        # Error: AttributeError (no tiene setter)

try:
    c.radio = -5
except ValueError as e:
    print(e)

Leyendo radio...
5
Asignando radio...
Leyendo radio...
10
Área: 314.15999999999997


No obstante, la forma más *pythonic* de controlar el acceso a la información es mediante **propiedades** (decoradores `@property` y `@*.setter`). 

Las propiedades permiten definir métodos que se comportan como atributos. Dicho de otro modo: permiten acceder desde fuera de la clase a los atributos del objeto bajo un reǵimen lógico determinado por los métodos decorados. 

In [ ]:
class Circulo:
    def __init__(self, radio):
        self.__radio = radio # Usamos guion bajo para el atributo interno

    @property
    def radio(self):
        """Comportamiento al leer (Getter)"""
        print("Leyendo radio...") # Por verbosidad nada más; evitar poner prints aquí!
        return self.__radio

    @radio.setter
    def radio(self, valor):
        """Comportamiento al escribir (Setter)"""
        if valor < 0:
            raise ValueError("El radio no puede ser negativo")
        print("Asignando radio...") # Por verbosidad nada más; evitar poner prints aquí!
        self.__radio = valor

    @property
    def area(self):
        """Propiedad de solo lectura (calculada)"""
        return 3.1416 * (self.__radio ** 2)

c = Circulo(5)
print(c.radio)       # Llama al getter radio(self)

c.radio = 10         # Llama al setter radio(self, valor)
print(c.radio)

print(c.area)        # Ahora area es una propiedad, no un método
# c.area = 20        # Error: AttributeError (area no tiene setter... ni tiene sentido que lo tenga)

try:
    c.radio = -5
except ValueError as e:
    print(e)

Leyendo radio...
5
Asignando radio...
Leyendo radio...
10
314.15999999999997
El radio no puede ser negativo


## Resumen
*   **Métodos de clase (`@classmethod`)**: Métodos que reciben la clase (`cls`). Típicamente para factorías de objetos.
*   **Métodos estáticos (`@staticmethod`)**: Métodos que no reciben `self` ni `cls`. Típicamente para funciones utilitarias.
*   **Visibilidad**: Por defecto, todo público. Se usa `_` delante para protegido (por convención), o `__` para privado (*name mangling*).
*   **Métodos getters y setters**: Permite encapsulamiento, lógica para controlar el acceso y manipulación de atributos privados.
*   **Propiedades (`@property`)**: Permite encapsulamiento 'pythónico', controlando getter y setter simulando acceso directo a atributos (`obj.propiedad`).